# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [ ]:
!pip install yelpapi
!pip install tqdm

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [ ]:
# Load API Credentials
with open('/Users/YOURUSERNAME/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [ ]:
# You can display the keys, but do NOT display the values!
login.keys()

In [ ]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

### Define Search Terms and File Paths


In [ ]:
# set our API call parameters and filename before the first call
location = 'Atlanta, GA 30033 '
term = 'mexican'

In [ ]:
# Programmatically obtain the city name only from the location string to use in our file name
location.split(',')[0]

In [ ]:
## Specify folder for saving data
FOLDER = 'Data/'
# This will make the Data folder for us if we don't have one
os.makedirs(FOLDER, exist_ok = True)


# Specifying JSON_FILE filename (can include a folder) to include the location and search term
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

#Verify the name of the JSON_FILE
JSON_FILE

In [ ]:
# We can extract the folder name from the file path 
folder = os.path.dirname(JSON_FILE)
folder

### Check if Json File exists and Create it if it doesn't

In [ ]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON folder name is not empty: (We have included a folder in our filepath)
    if len(folder)>0:
        # create the folder (In our case, the folder is Data)
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Crating and saving empty list to file.")

    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

### Load JSON FIle and account for previous results

### Make the first API call to explore the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [ ]:
# use our yelp_api variable's search_query method to perform our API call
# for help with any function, go inside () and click shift + tab
# we MUST include location (or lat/long) but we can include other arguments as well




In [ ]:
# Check data type of the results


In [ ]:
# Check the length of the results


In [ ]:
# List the keys of the dictionary


In [ ]:
# Explore each key


In [ ]:
# Explore each key


In [ ]:
# Explore each key


- Where is the actual data we want to save?

In [ ]:
# What is the datatype for the businesses?


In [ ]:
# Display first result from the list of dictionaries

In [ ]:
## We can convert the a list of dictionaries  to a datafram


In [ ]:
## How many did we get the details for?


- Calculate how many pages of results needed to cover the total_results

In [ ]:
# How many pages are needed?


In [ ]:
# Use math.ceil to round up for the total number of pages of results.


# Create the JSON FILE, give it a programmatic name
# Save our first page of results to the JSON file

In [ ]:
## Read in results in progress file and check the length
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

In [ ]:
# Preview the contents of the json file (hint, we expect this to be empty at this point)


In [ ]:
# Determine how many results are present already (hint, we expect this to be 0 at this point

In [ ]:
# Make a loop to extract data from each page and save it to the JSON_FILE

for i in tqdm_notebook(range(n_pages)): 
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
     
       
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results to use as offset for query
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location = location, term = term, offset = n_results)  # Notice we added the offset to our search_query
        

        ## append new results and save to file  (extend is similar to append, but adds a list to a list)
        previous_results.extend(results['businesses'])

        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.

    except Exception as e:
        print(' [!] ERROR', e)

## Open the Final JSON File with Pandas

In [ ]:
# Read the json file in as a df
df = pd.read_json(JSON_FILE)

In [ ]:
df.head()

In [ ]:
# Verify the .info 
df.info()

In [ ]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

In [ ]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

## Next Class: Processing the Results and Mapping 